In [63]:
import os
import sys
import cv2
import tensorflow as tf
import numpy as np
from tensorflow.keras import Model
from tensorflow.keras.layers import Add, Concatenate, Conv2D, Input, Lambda, LeakyReLU, MaxPool2D, UpSampling2D, ZeroPadding2D, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import binary_crossentropy, sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, TensorBoard

import tensorflow_datasets as tfds
import emergency_dataset

In [64]:
ANCHORS = np.array([
    (10, 14), (23, 27), (37, 58),
    (81, 82), (135, 169),  (344, 319)], np.float32) / 416

MASKS = np.array([[3, 4, 5], [0, 1, 2]])

LAYERS = [
    'yolo_darknet',
    'yolo_conv_0',
    'yolo_output_0',
    'yolo_conv_1',
    'yolo_output_1',
]

SIZE = 416
CLASSES = 1
LR = 1e-4
EPOCHS = 100

In [65]:
def DarknetConv(x, filters, size, strides=1, batch_norm=True):
    if strides == 1:
        padding = 'same'
    else:
        x = ZeroPadding2D(((1,0), (1,0)))(x)
        padding = 'valid'
    x = Conv2D(filters, size, strides, padding, use_bias= not batch_norm, kernel_regularizer=l2(0.0005))(x)
    if batch_norm:
        x = BatchNormalization()(x)
        x = LeakyReLU(0.1)(x)
    return x

def DarknetTiny(name=None):
    x = inputs = Input([None, None, 3])
    x = DarknetConv(x, 16, 3)
    x = MaxPool2D(2, 2, 'same')(x)
    x = DarknetConv(x, 32, 3)
    x = MaxPool2D(2, 2, 'same')(x)
    x = DarknetConv(x, 64, 3)
    x = MaxPool2D(2, 2, 'same')(x)
    x = DarknetConv(x, 128, 3)
    x = MaxPool2D(2, 2, 'same')(x)
    x = x_8 = DarknetConv(x, 256, 3)
    x = MaxPool2D(2, 2, 'same')(x)
    x = DarknetConv(x, 512, 3)
    x = MaxPool2D(2, 1, 'same')(x)
    x = DarknetConv(x, 1024, 3)
    return tf.keras.Model(inputs, (x_8, x), name=name)

In [66]:
def YoloConvTiny(filters, name=None):
    def yolo_conv(x_in):
        if isinstance(x_in, tuple):
            inputs = Input(x_in[0].shape[1:]), Input(x_in[1].shape[1:])
            x, x_skip = inputs

            # concat with skip connection
            x = DarknetConv(x, filters, 1)
            x = UpSampling2D(2)(x)
            x = Concatenate()([x, x_skip])
        else:
            x = inputs = Input(x_in.shape[1:])
            x = DarknetConv(x, filters, 1)

        return Model(inputs, x, name=name)(x_in)
    return yolo_conv

def YoloOutput(filters, anchors, classes, name=None):
    def yolo_output(x_in):
        x = inputs = Input(x_in.shape[1:])
        x = DarknetConv(x, filters * 2, 3)
        x = DarknetConv(x, anchors * (classes + 5), 1, batch_norm=False)
        x = Lambda(lambda x: tf.reshape(x, (-1, tf.shape(x)[1], tf.shape(x)[2],
                                            anchors, classes + 5)))(x)
        return tf.keras.Model(inputs, x, name=name)(x_in)
    return yolo_output

In [67]:
def _meshgrid(n_a, n_b):
    return [
        tf.reshape(tf.tile(tf.range(n_a), [n_b]), (n_b, n_a)),
        tf.reshape(tf.repeat(tf.range(n_b), n_a), (n_b, n_a))
    ]

In [68]:
def yolo_boxes(pred, anchors, classes):
    
    grid_size = tf.shape(pred)[1:3]
    
    box_xy, box_wh, objectness, class_probs = tf.split(pred, (2,2,1,classes), axis=-1)
    box_xy = tf.sigmoid(box_xy)
    objectness = tf.sigmoid(objectness)
    class_probs = tf.sigmoid(class_probs)
    pred_box = tf.concat((box_xy, box_wh), axis=-1)
    
    grid = _meshgrid(grid_size[1], grid_size[0])
    grid = tf.expand_dims(tf.stack(grid, axis=-1), axis=2)
    
    box_xy = (box_xy + tf.cast(grid, tf.float32)) / tf.cast(grid_size, tf.float32)
    box_wh = tf.exp(box_wh) * anchors
    
    box_x1y1 = box_xy - box_wh / 2
    box_x2y2 = box_xy + box_wh / 2
    bbox = tf.concat([box_x1y1, box_x2y2], axis=-1)
    
    return bbox, objectness, class_probs, pred_box

def yolo_nms(outputs, anchors, masks, classes):
    
    b,c,t = [],[],[]
    for o in outputs:
        b.append(tf.reshape(o[0], (tf.shape(o[0])[0], -1, tf.shape(o[0])[-1])))
        c.append(tf.reshape(o[1], (tf.shape(o[1])[0], -1, tf.shape(o[1])[-1])))
        t.append(tf.reshape(o[2], (tf.shape(o[2])[0], -1, tf.shape(o[2])[-1])))
    
    bbox = tf.concat(b, axis=1)
    confidence = tf.concat(c, axis=1)
    class_probs = tf.concat(t, axis=1)
    
    if classes == 1:
        scores = confidence
    else:
        scores = confidence * class_probs
    
    dscores = tf.squeeze(scores, axis=0)
    scores = tf.reduce_max(dscores, [1])
    bbox = tf.reshape(bbox, (-1, 4))
    classes = tf.argmax(dscores, 1)
    selected_indices, selected_scores = tf.image.non_max_suppression_with_scores(
        boxes=bbox,
        scores=scores,
        max_output_size=100,
        iou_threshold=0.5,
        score_threshold=0.5,
        soft_nms_sigma=0.5
    )
    
    num_valid_nms_boxes = tf.shape(selected_indices)[0]
    selected_indices = tf.concat([selected_indices, tf.zeros(100 - num_valid_nms_boxes, tf.int32)], 0)
    selected_scores = tf.concat([selected_scores, tf.zeros(100 - num_valid_nms_boxes, tf.float32)], -1)
    
    boxes = tf.gather(bbox, selected_indices)
    boxes = tf.expand_dims(boxes, axis=0)
    scores = selected_scores
    scores = tf.expand_dims(scores, axis=0)
    classes = tf.gather(classes, selected_indices)
    classes = tf.expand_dims(classes, axis=0)
    valid_detections = num_valid_nms_boxes
    valid_detections = tf.expand_dims(valid_detections, axis=0)
    
    return boxes, scores, classes, valid_detections

In [69]:
def YoloTiny(size=None, channels=3, anchors=ANCHORS, masks=MASKS, classes=CLASSES, training=False):
    x = inputs = Input([size, size, channels], name='input')

    x_8, x = DarknetTiny(name='yolo_darknet')(x)

    x = YoloConvTiny(256, name='yolo_conv_0')(x)
    output_0 = YoloOutput(256, len(masks[0]), classes, name='yolo_output_0')(x)
    
    x = YoloConvTiny(128, name='yolo_conv_1')((x, x_8))
    output_1 = YoloOutput(128, len(masks[1]), classes, name='yolo_output_1')(x)

    if training:
        return Model(inputs, (output_0, output_1), name='yolov3')

    boxes_0 = Lambda(lambda x: yolo_boxes(x, anchors[masks[0]], classes),
                     name='yolo_boxes_0')(output_0)
    boxes_1 = Lambda(lambda x: yolo_boxes(x, anchors[masks[1]], classes),
                     name='yolo_boxes_1')(output_1)
    outputs = Lambda(lambda x: yolo_nms(x, anchors, masks, classes),
                     name='yolo_nms')((boxes_0[:3], boxes_1[:3]))
    return Model(inputs, outputs, name='yolov3_tiny')

In [70]:
def broadcast_iou(box1, box2):
    box1 = tf.expand_dims(box1, -2)
    box2 = tf.expand_dims(box2, 0)
    
    new_shape = tf.broadcast_dynamic_shape(tf.shape(box1), tf.shape(box2))
    box1 = tf.broadcast_to(box1, new_shape)
    box2 = tf.broadcast_to(box2, new_shape)
    
    int_w = tf.maximum(tf.minimum(box1[..., 2], box2[..., 2]) - tf.maximum(box1[..., 0], box2[..., 0]), 0)
    int_h = tf.maximum(tf.minimum(box1[..., 3], box2[..., 3]) - tf.maximum(box1[..., 1], box2[..., 1]), 0)
    int_area = int_w * int_h
    
    box1_area = (box1[..., 2] - box1[..., 0]) * (box1[..., 3] - box1[..., 1])
    box2_area = (box2[..., 2] - box2[..., 0]) * (box2[..., 3] - box2[..., 1])
    
    return int_area / (box1_area + box2_area - int_area)

def YoloLoss(anchors, classes=CLASSES, ignore_thresh=0.5):
    def yolo_loss(y_true, y_pred):
        
        pred_box, pred_obj, pred_class, pred_xywh = yolo_boxes(y_pred, anchors, classes)
        pred_xy = pred_xywh[..., 0:2]
        pred_wh = pred_xywh[..., 2:4]
        
        true_box, true_obj, true_class_idx = tf.split(y_true, (4,1,1), axis=-1)
        true_xy = (true_box[..., 0:2] + true_box[..., 2:4]) / 2
        true_wh = true_box[..., 2:4] - true_box[..., 0:2]
        box_loss_scale = 2 - true_wh[..., 0] * true_wh[..., 1]
        
        grid_size = tf.shape(y_true)[1]
        grid = tf.meshgrid(tf.range(grid_size), tf.range(grid_size))
        grid = tf.expand_dims(tf.stack(grid, axis=-1), axis=2)
        true_xy = true_xy * tf.cast(grid_size, tf.float32) - tf.cast(grid, tf.float32)
        true_wh = tf.math.log(true_wh / anchors)
        true_wh = tf.where(tf.math.is_inf(true_wh), tf.zeros_like(true_wh), true_wh)
        
        obj_mask = tf.squeeze(true_obj, -1)
        best_iou = tf.map_fn(lambda x : tf.reduce_max(broadcast_iou(x[0], tf.boolean_mask(x[1], tf.cast(x[2], tf.bool))), axis=-1), (pred_box, true_box, obj_mask), tf.float32)
        ignore_mask = tf.cast(best_iou < ignore_thresh, tf.float32)
        
        xy_loss = obj_mask * box_loss_scale * tf.reduce_sum(tf.square(true_xy - pred_xy), axis=-1)
        wh_loss = obj_mask * box_loss_scale * tf.reduce_sum(tf.square(true_wh - pred_wh), axis=-1)
        obj_loss = binary_crossentropy(true_obj, pred_obj)
        obj_loss = obj_mask * obj_loss + (1 - obj_mask) * ignore_mask * obj_loss
        class_loss = obj_mask * sparse_categorical_crossentropy(true_class_idx, pred_class)
        
        xy_loss = tf.reduce_sum(xy_loss, axis=(1,2,3))
        wh_loss = tf.reduce_sum(wh_loss, axis=(1,2,3))
        obj_loss = tf.reduce_sum(obj_loss, axis=(1,2,3))
        class_loss = tf.reduce_sum(class_loss, axis=(1,2,3))
        
        return xy_loss + wh_loss + obj_loss + class_loss
    return yolo_loss

In [71]:
def convert_darknet_weigths(weight_file, output_file):
    
    model = YoloTiny(classes=80)
    
    wf = open(weight_file, 'rb')
    _ = np.fromfile(wf, dtype=np.int32, count=5)
    
    for layer_name in LAYERS:
        submodel = model.get_layer(layer_name)
        for i, layer in enumerate(submodel.layers):
            if not layer.name.startswith('conv2d'):
                continue
            
            batch_norm = None
            if i + 1 < len(submodel.layers) and submodel.layers[i+1].name.startswith('batch_norm'):
                batch_norm = submodel.layers[i+1]
            
            filters = layer.filters
            size = layer.kernel_size[0]
            in_dim = layer.get_input_shape_at(0)[-1]
            
            if batch_norm is None:
                conv_bias = np.fromfile(wf, dtype=np.float32, count=filters)
            else:
                bn_weights = np.fromfile(wf, dtype=np.float32, count = 4*filters)
                bn_weights = bn_weights.reshape((4, filters))[[1,0,2,3]]
            
            conv_shape = (filters, in_dim, size, size)
            conv_weights = np.fromfile(wf, dtype=np.float32, count=np.product(conv_shape))
            conv_weights = conv_weights.reshape(conv_shape).transpose([2,3,1,0])
            
            if batch_norm is None:
                layer.set_weights([conv_weights, conv_bias])
            else:
                layer.set_weights([conv_weights])
                batch_norm.set_weights(bn_weights)
        
    wf.close()

    model.save_weights(output_file)

def draw_outputs(img, outputs):
    boxes, objectness, classes, nums = outputs
    boxes, objectness, classes, nums = boxes[0], objectness[0], classes[0], nums[0]
    wh = np.flip(img.shape[0:2])
    for i in range(nums):
        if objectness[i] <= 0:
            continue
        x1y1 = tuple((np.array(boxes[i][0:2]) * wh).astype(np.int32))
        x2y2 = tuple((np.array(boxes[i][2:4]) * wh).astype(np.int32))
        img = cv2.rectangle(img, x1y1, x2y2, (255,0,0), 2)
    return img

In [72]:
def transform_images(im, size):
    return tf.image.resize(im, (size, size)) / 255

@tf.function
def transform_labels_for_output(label, grid_size, anchor_idxs):
    
    N = tf.shape(label)[0]
    y_out = tf.zeros((N, grid_size, grid_size, tf.shape(anchor_idxs)[0], 6))
    
    anchor_idxs = tf.cast(anchor_idxs, tf.int32)
    
    indexes = tf.TensorArray(tf.int32, 1, dynamic_size=True)
    updates = tf.TensorArray(tf.float32, 1, dynamic_size=True)
    
    idx = 0
    for i in tf.range(N):
        for j in tf.range(tf.shape(label)[1]):
            if tf.equal(label[i][j][2], 0):
                continue
            
            anchor_eq = tf.equal(anchor_idxs, tf.cast(label[i][j][5], tf.int32))
            if tf.reduce_any(anchor_eq):
                box = label[i][j][0:4]
                box_xy = (label[i][j][0:2] + label[i][j][2:4]) / 2
                
                anchor_idx = tf.cast(tf.where(anchor_eq), tf.int32)
                grid_xy = tf.cast(box_xy // (1/grid_size), tf.int32)
                
                indexes = indexes.write(idx, [i, grid_xy[1], grid_xy[0], anchor_idx[0][0]])
                updates = updates.write(idx, [box[0], box[1], box[2], box[3], 1, label[i][j][4]])
                
                idx += 1
    
    return tf.tensor_scatter_nd_update(y_out, indexes.stack(), updates.stack())
                

def transform_labels(label, size):
    
    # reorder, format label
    c = tf.expand_dims(label[..., 0], axis=-1)
    x1y1 = label[..., 1:3] - label[..., 3:5] / 2
    x2y2 = label[..., 1:3] + label[..., 3:5] / 2
    label = tf.concat((x1y1, x2y2, c), axis=-1)
    
    y_outs = []
    grid_size = size // 32
    
    anchors = tf.cast(ANCHORS, tf.float32)
    anchor_area = anchors[..., 0] * anchors[..., 1]
    
    box_wh = label[..., 2:4] - label[..., 0:2]
    box_wh = tf.tile(tf.expand_dims(box_wh, -2), (1, 1, tf.shape(anchors)[0], 1))
    box_area = box_wh[..., 0] * box_wh[..., 1]
    
    intersection = tf.minimum(box_wh[..., 0], anchors[..., 0]) * tf.minimum(box_wh[..., 1], anchors[..., 1])
    iou = intersection / (box_area + anchor_area - intersection)
    
    anchor_idx = tf.cast(tf.argmax(iou, axis=-1), tf.float32)
    anchor_idx = tf.expand_dims(anchor_idx, axis=-1)
    
    label = tf.concat([label, anchor_idx], axis=-1)
    
    for anchor_idxs in MASKS:
        y_outs.append(transform_labels_for_output(label, grid_size, anchor_idxs))
        grid_size *= 2
    
    return tuple(y_outs)

In [73]:
def recursive_freeze(model):
    model.trainable = False
    if isinstance(model, Model):
        for layer in model.layers:
            recursive_freeze(layer)

def setup():
    
    model = YoloTiny(size=SIZE, training=True)
    
    model_pretrained = YoloTiny(size=SIZE, training=True, classes=80)
    model_pretrained.load_weights("./yolov3_tiny.tf")
    model.get_layer("yolo_darknet").set_weights(model_pretrained.get_layer("yolo_darknet").get_weights())
    recursive_freeze(model.get_layer("yolo_darknet"))
    
    optimizer = Adam(lr=LR)
    loss = [YoloLoss(ANCHORS[mask]) for mask in MASKS]
    
    model.compile(optimizer=optimizer, loss=loss)
    
    return model, optimizer, loss

def train(chk=0):
    
    if chk == 0:
        model, optimizer, loss = setup()
    else:
        model = YoloTiny(size=SIZE, training=True)
        model.load_weights("./checkpointsTiny/train_" + str(chk) + ".tf")
        recursive_freeze(model.get_layer("yolo_darknet"))
        optimizer = Adam(learning_rate=LR)
        loss = [YoloLoss(ANCHORS[mask]) for mask in MASKS]
        model.compile(optimizer=optimizer, loss=loss)
    
    train_dataset = tfds.load("emergency_dataset", split='train', as_supervised=True)
    train_dataset = train_dataset.shuffle(1000)
    train_dataset = train_dataset.batch(8)
    #train_dataset = train_dataset.map(lambda x: {"image" : transform_images(x["image"], 416), "label" : transform_labels(x["label"], 416)})
    train_dataset = train_dataset.map(lambda x, y : (transform_images(x, 416), transform_labels(y, 416)))
    
    val_dataset = tfds.load("emergency_dataset", split='val', as_supervised=True)
    val_dataset = val_dataset.shuffle(1000)
    val_dataset = val_dataset.batch(8)
    #val_dataset = val_dataset.map(lambda x: {"image" : transform_images(x["image"], 416), "label" : transform_labels(x["label"], 416)})
    val_dataset = val_dataset.map(lambda x, y : (transform_images(x, 416), transform_labels(y, 416)))
    
    callbacks = [
        ReduceLROnPlateau(verbose=1),
        # EarlyStopping(patience=3, verbose=1),
        ModelCheckpoint('./checkpointsTiny/train_{epoch}.tf', verbose=1, save_weights_only=True),
        TensorBoard(log_dir='logs')
    ]
    
    history = model.fit(train_dataset,
                        epochs=EPOCHS,
                        callbacks=callbacks,
                        validation_data=val_dataset)

In [74]:
train(100)

Epoch 1/100
117/117 [==============================] - ETA: 0s - loss: 4.2386 - yolo_output_0_loss: 0.1714 - yolo_output_1_loss: 0.3766
Epoch 1: saving model to ./checkpointsTiny\train_1.tf
117/117 [==============================] - 10s 55ms/step - loss: 4.2386 - yolo_output_0_loss: 0.1714 - yolo_output_1_loss: 0.3766 - val_loss: 10.3606 - val_yolo_output_0_loss: 6.3562 - val_yolo_output_1_loss: 0.3138 - lr: 1.0000e-09
Epoch 2/100
117/117 [==============================] - ETA: 0s - loss: 4.2470 - yolo_output_0_loss: 0.1704 - yolo_output_1_loss: 0.3860
Epoch 2: saving model to ./checkpointsTiny\train_2.tf
117/117 [==============================] - 6s 49ms/step - loss: 4.2470 - yolo_output_0_loss: 0.1704 - yolo_output_1_loss: 0.3860 - val_loss: 10.3552 - val_yolo_output_0_loss: 6.3524 - val_yolo_output_1_loss: 0.3122 - lr: 1.0000e-09
Epoch 3/100
117/117 [==============================] - ETA: 0s - loss: 4.2389 - yolo_output_0_loss: 0.1711 - yolo_output_1_loss: 0.3772
Epoch 3: saving mod

In [47]:
model = YoloTiny(training=False)
model.load_weights("./checkpointsTiny/train_40.tf")

In [48]:
ds = tfds.load("emergency_dataset", split="val")
ds = ds.batch(1)
ds = ds.shuffle(1000)
ds = ds.take(1)
im = 1
for e in ds:
    im = e['image']
imraw = im
im = transform_images(im, 416)

In [77]:
def export_tflite(inpath, outpath):
    model = YoloTiny(size=SIZE)
    model.load_weights(inpath)
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.experimental_new_converter = True
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_types = [tf.float16]
    tflite_model = converter.convert()
    f = open(outpath, 'wb')
    f.write(tflite_model)
    f.close()

In [ ]:
export_tflite("./checkpointsTiny/train_100.tf", "./models/yoloTiny.tflite")

In [30]:
def detect_image(model_path, img_path):
    interpreter = tf.lite.Interpreter(model_path)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    imRaw = cv2.imread(img_path)
    im = cv2.cvtColor(imRaw, cv2.COLOR_BGR2RGB)
    im = tf.convert_to_tensor(im)
    im = tf.expand_dims(im, 0)
    im = transform_images(im, 416)
    
    interpreter.set_tensor(input_details[0]['index'], im)
    interpreter.invoke()
    boxes = interpreter.get_tensor(output_details[2]['index'])
    scores = interpreter.get_tensor(output_details[3]['index'])
    classes = interpreter.get_tensor(output_details[0]['index'])
    nums = interpreter.get_tensor(output_details[1]['index'])
    imRaw = draw_outputs(imRaw, (boxes, scores, classes, nums))
    cv2.imshow("a", imRaw)
    cv2.waitKey(0)
    
def detect_video(model_path, vid_path):
    interpreter = tf.lite.Interpreter(model_path)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    cap = cv2.VideoCapture(vid_path)
    while cap.isOpened():
        ret, frameRaw = cap.read()
        frame = cv2.cvtColor(frameRaw, cv2.COLOR_BGR2RGB)
        frame = tf.convert_to_tensor(frame)
        frame = tf.expand_dims(frame, 0)
        frame = transform_images(frame, 416)
        
        interpreter.set_tensor(input_details[0]['index'], frame)
        interpreter.invoke()
        boxes = interpreter.get_tensor(output_details[2]['index'])
        scores = interpreter.get_tensor(output_details[3]['index'])
        classes = interpreter.get_tensor(output_details[0]['index'])
        nums = interpreter.get_tensor(output_details[1]['index'])
        
        frameRaw = draw_outputs(frameRaw, (boxes, scores, classes, nums))
        cv2.imshow("a", frameRaw)
        cv2.waitKey(1)
    
    cap.release()
    cv2.destroyAllWindows()

In [34]:
# i = 756
# for filename in os.listdir("/Users/stack/Downloads/imgs/"):
#     impath = os.path.join("/Users/stack/Downloads/imgs/", filename)
#     labelfilename = filename[:-4] + '.txt'
#     labelpath = os.path.join("/Users/stack/Downloads/lbls", labelfilename)
#     im = cv2.imread(impath)
#     im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
#     im = cv2.cvtColor(im, cv2.COLOR_GRAY2BGR)
#     im = cv2.resize(im, (512, 512))
#     cv2.imwrite(os.path.join("/Users/stack/Documents/warudo/yolo/newData/images", str(i) + ".jpg"), im)
#     a = np.loadtxt(labelpath)
#     a = np.reshape(a, (-1, 5))
#     for k in range(100 - a.shape[0]):
#         a = np.concatenate((a, np.array([[0.0, 0.0, 0.0, 0.0, 0.0]])), axis=0)
#     np.savetxt((os.path.join("/Users/stack/Documents/warudo/yolo/newData/labels", str(i) + ".txt")), a)
#     i += 1